In [111]:
from pathlib import Path
import re
import json

import pandas as pd

from utils import read_last_line_from_file

In [112]:
OUTPUT_DIR = Path("output")

In [127]:
class Run:
    def __init__(self, log_file: Path):
        if not log_file.exists():
            raise ValueError(f"log file `{log_file}` does not exist")
        self.log_file = log_file
    
    def seed(self) -> int:
        seed = re.findall(r".*_(\d+).log", str(self.log_file))
        assert len(seed) == 1
        seed = int(seed[0])
        return seed

    def metrics(self) -> dict[str, float]:
        last_line = read_last_line_from_file(self.log_file)
        _date, metrics_string = last_line.split(" - ", maxsplit=1)
        metrics_string = metrics_string.replace("'", "\"")
        metrics = json.loads(metrics_string)
        metrics = {k: float(v) for k, v in metrics.items()}

        del metrics["Epoch"]
        return metrics


class SeededRuns:
    def __init__(
        self,
        runs: list[Run],
        model_name = "Model",
        dataset_name = None,
    ):
        self.runs = runs
        self.model_name = model_name
        self.dataset = dataset_name

    @classmethod
    def from_model_name_and_dataset(
        cls, model_name: str, dataset: str, output_dir: Path = OUTPUT_DIR, log_file_extension: str = "log",
    ):
        model_output_path = output_dir / model_name
        if not model_output_path.exists():
            raise ValueError(f"model output path `{model_output_path}` does not exist")

        seeded_runs = model_output_path.glob(f"{model_name}_{dataset}_*.{log_file_extension}")
        seeded_runs = [Run(run) for run in seeded_runs]
        return cls(seeded_runs, model_name, dataset)
    
    def seeds(self):
        return [run.seed() for run in self.runs]
    
    def metrics_df(self):
        metrics = [run.metrics() for run in self.runs]
        return pd.DataFrame(metrics)
    
    def describe_metrics_df(self, *, caption: bool = False):
        metrics_df = self.metrics_df()
        metrics_df = metrics_df.describe().loc[["mean", "std", "min", "max"]]
        if caption:
            caption = f"{self.model_name} performance on {self.dataset} over {len(self.seeds())} seeds"
            metrics_df = metrics_df.style.set_caption(caption)

        return metrics_df

In [128]:
# Models on the LastFM dataset
bert4rec_lastfm = SeededRuns.from_model_name_and_dataset("BERT4Rec", "LastFM")
sasrec_lastfm = SeededRuns.from_model_name_and_dataset("SASRec", "LastFM")
duorec_lastfm = SeededRuns.from_model_name_and_dataset("DuoRec", "LastFM")
fearec_lastfm = SeededRuns.from_model_name_and_dataset("FEARec", "LastFM")
bsarec_lastfm = SeededRuns.from_model_name_and_dataset("BSARec", "LastFM")

# Models on the Diginetica dataset
# Diginetica models
bert4rec_diginetica = SeededRuns.from_model_name_and_dataset("BERT4Rec", "Diginetica")
sasrec_diginetica = SeededRuns.from_model_name_and_dataset("SASRec", "Diginetica")
duorec_diginetica = SeededRuns.from_model_name_and_dataset("DuoRec", "Diginetica")
fearec_diginetica = SeededRuns.from_model_name_and_dataset("FEARec", "Diginetica")
bsarec_diginetica = SeededRuns.from_model_name_and_dataset("BSARec", "Diginetica")

In [134]:
display(bert4rec_lastfm.describe_metrics_df(caption=True))
display(sasrec_lastfm.describe_metrics_df(caption=True))
display(duorec_lastfm.describe_metrics_df(caption=True))
display(fearec_lastfm.describe_metrics_df(caption=True))
display(bsarec_lastfm.describe_metrics_df(caption=True))

,HR@5,NDCG@5,HR@10,NDCG@10,HR@20,NDCG@20
mean,0.029740,0.018700,0.049720,0.025120,0.079820,0.032660
std,0.005387,0.003130,0.005335,0.002687,0.005258,0.002608
min,0.023900,0.015200,0.043100,0.021500,0.070600,0.028500
max,0.034900,0.021600,0.057800,0.028500,0.083500,0.034900


,HR@5,NDCG@5,HR@10,NDCG@10,HR@20,NDCG@20
mean,0.039440,0.026820,0.059840,0.033360,0.088620,0.040640
std,0.002057,0.001705,0.002285,0.001547,0.004764,0.002239
min,0.036700,0.025200,0.056000,0.031100,0.080700,0.037400
max,0.042200,0.028900,0.061500,0.035000,0.092700,0.042800


,HR@5,NDCG@5,HR@10,NDCG@10,HR@20,NDCG@20
mean,0.042400,0.032040,0.057960,0.037020,0.089540,0.044880
std,0.002094,0.000666,0.004505,0.000912,0.005773,0.001112
min,0.040400,0.031200,0.054100,0.035800,0.083500,0.043400
max,0.045900,0.032900,0.065100,0.038200,0.096300,0.046000


,HR@5,NDCG@5,HR@10,NDCG@10,HR@20,NDCG@20
mean,0.042000,0.029100,0.058920,0.034520,0.085320,0.041080
std,0.008903,0.005377,0.011804,0.006351,0.014559,0.007007
min,0.028400,0.019900,0.040400,0.023800,0.062400,0.029300
max,0.050500,0.032900,0.072500,0.040600,0.101800,0.047900


,HR@5,NDCG@5,HR@10,NDCG@10,HR@20,NDCG@20
mean,0.049920,0.033900,0.073400,0.041440,0.107700,0.049980
std,0.002643,0.001687,0.004270,0.001372,0.005200,0.001728
min,0.046800,0.031900,0.067900,0.039400,0.102800,0.047400
max,0.053200,0.035900,0.078000,0.042900,0.116500,0.051900


In [135]:
display(bert4rec_diginetica.describe_metrics_df(caption=True))
display(sasrec_diginetica.describe_metrics_df(caption=True))
display(duorec_diginetica.describe_metrics_df(caption=True))
display(fearec_diginetica.describe_metrics_df(caption=True))
display(bsarec_diginetica.describe_metrics_df(caption=True))

,HR@5,NDCG@5,HR@10,NDCG@10,HR@20,NDCG@20
mean,0.121000,0.077480,0.190200,0.099740,0.277260,0.121720
std,0.001042,0.000936,0.002084,0.001228,0.001397,0.000942
min,0.119600,0.076400,0.187200,0.098500,0.276000,0.120600
max,0.122200,0.078700,0.192400,0.101300,0.279300,0.122800


,HR@5,NDCG@5,HR@10,NDCG@10,HR@20,NDCG@20
mean,0.103940,0.066520,0.165700,0.086360,0.250480,0.107740
std,0.001282,0.000303,0.003043,0.001272,0.004563,0.001626
min,0.103000,0.066000,0.162100,0.085000,0.246700,0.106300
max,0.106200,0.066700,0.169500,0.087900,0.258100,0.110300


,HR@5,NDCG@5,HR@10,NDCG@10,HR@20,NDCG@20
mean,0.151960,0.099440,0.227900,0.123860,0.322080,0.147620
std,0.001150,0.000876,0.001042,0.000623,0.001117,0.000756
min,0.150700,0.098300,0.226500,0.122900,0.320500,0.146400
max,0.153200,0.100200,0.229000,0.124500,0.323400,0.148200


,HR@5,NDCG@5,HR@10,NDCG@10,HR@20,NDCG@20
mean,0.153460,0.100560,0.231760,0.125740,0.327100,0.149800
std,0.001339,0.000695,0.001670,0.000832,0.002192,0.000316
min,0.151500,0.099800,0.229900,0.124900,0.324300,0.149500
max,0.155200,0.101400,0.234200,0.126700,0.329200,0.150300


,HR@5,NDCG@5,HR@10,NDCG@10,HR@20,NDCG@20
mean,0.156280,0.101960,0.233380,0.126700,0.325620,0.150000
std,0.001555,0.000805,0.001316,0.000624,0.001695,0.000791
min,0.154600,0.100800,0.231900,0.125800,0.323600,0.149000
max,0.158800,0.102900,0.235300,0.127500,0.327400,0.150800
